In [1]:
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
import combat_probability as cp
from icepool import d

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True

COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [51]:
# Fig. 1: plots the probability of PCs and monsters winning for a range of difficulties

difficulties = list(np.linspace(0.01, 1.0, 21))

pc_win_pct = []
mon_win_pct = []
for diff in difficulties:
    party_hp = 100
    pc_r_win = 3
    pc_dpr_cv = 0.5

    mon_hp = 100
    mon_r_win = pc_r_win/diff
    mon_dpr_cv = 0.5


    party_dpr = mon_hp/pc_r_win
    party_size = 4
    pc_hp = party_hp/party_size
    pc_dpr = party_dpr/party_size

    mon_dpr = party_hp/mon_r_win

    # construct combatants
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='win probability',
            range=[-0.01,1.01],
            tickformat='.1f',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=0.98, y=0.98,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    """fig.add_trace(go.Scatter(
        x=[diff,diff],
        y=[0,1],
        showlegend=False,
        mode='lines',
        line=dict(
            color='black', 
            width=1,
            dash='dash'
        ),
        hoverinfo='none',
    ))"""

    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        #line_color=color,
        #marker_color=color,
        fillcolor=color,
        #hoveron = 'points+fills', # select where hover is active
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.50,
        text=name,
        showarrow=False,
    )

# plot data
fig.add_trace(go.Scatter(
    x=difficulties,
    y=pc_win_pct,
    name=f'PCs',
    mode='lines',
    line_color=COLOR_LIST[0],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=difficulties,
    y=mon_win_pct,
    name=f'NPCs',
    mode='lines',
    line_color=COLOR_LIST[1],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-diff-deadly-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-diff-deadly-small')

In [52]:
# plots the average damage taken by the PCs and monsters for a range of difficulties

difficulties = list(np.linspace(0.05, 1.0, 21))
pc_win_pct = []
mon_win_pct = []
pc_hp_loss = []
pc_hp_loss_sigma = []
mon_hp_loss = []
mon_hp_loss_sigma = []
for diff in difficulties:
    party_hp = 100
    pc_r_win = 3
    pc_dpr_cv = 0.5

    mon_hp = 100
    mon_r_win = pc_r_win/diff
    mon_dpr_cv = 0.5


    party_dpr = mon_hp/pc_r_win
    party_size = 4
    pc_hp = party_hp/party_size
    pc_dpr = party_dpr/party_size

    mon_dpr = party_hp/mon_r_win

    
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

    pc_hp_loss += [e.group_damage_mean('NPCs', method='approx', clip=True)/(party_size*pc_hp)]
    pc_hp_loss_sigma += [e.group_damage_sigma('NPCs', method='approx', clip=True)/(party_size*pc_hp)]

    mon_hp_loss += [e.group_damage_mean('PCs', method='approx', clip=True)/(mon_hp)]
    mon_hp_loss_sigma += [e.group_damage_sigma('PCs', method='approx', clip=True)/(mon_hp)]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='damage taken / hit points',
            range=[-0.01,1.01],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=0.98, y=0.98,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    """fig.add_trace(go.Scatter(
        x=[diff,diff],
        y=[0,1],
        showlegend=False,
        mode='lines',
        line=dict(
            color='black', 
            width=1,
            dash='dash'
        ),
        hoverinfo='none',
    ))"""

    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        #line_color=color,
        #marker_color=color,
        fillcolor=color,
        #hoveron = 'points+fills', # select where hover is active
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.5,
        text=name,
        showarrow=False,
    )

# plot data
fig.add_trace(go.Scatter(
    x=difficulties,
    y=pc_hp_loss,
    name=f'PCs',
    mode='lines',
    line_color=COLOR_LIST[0],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'HP lost %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=difficulties,
    y=mon_hp_loss,
    name=f'NPCs',
    mode='lines',
    line_color=COLOR_LIST[1],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'HP lost %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=[0,1],
    y=[0,1],
    name=f'1-to-1',
    showlegend=False,
    mode='lines',
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-diff-deadly-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-diff-deadly-small')

In [53]:
# Fig. 2: plots the probability of PCs and monsters winning for a range of monster CV values
diff = 0.7 # deadly

party_hp = 100
pc_r_win = 3
pc_dpr_cv = 0.5


mon_hp = 100
mon_r_win = pc_r_win/diff
mon_dpr_cvs = np.linspace(0.2, 0.8, 7)


party_dpr = mon_hp/pc_r_win
party_size = 4
pc_hp = party_hp/party_size
pc_dpr = party_dpr/party_size

mon_dpr = party_hp/mon_r_win


# original
pc_win_pct = []
mon_win_pct = []
for mon_dpr_cv in mon_dpr_cvs:
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]
    #print(e.encounter_length(units='rounds', method='approx'))


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='NPC damage sigma / mean',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='win probability',
            range=[0.0,1],
            tickformat='.1f',
            tick0=0, dtick=0.20,
            minor=dict(tick0=0, dtick=0.10),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=pc_win_pct,
    name=f'PCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=mon_win_pct,
    name=f'NPCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-npc-cv-deadly-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-npc-cv-deadly-small')

In [54]:
# Fig. 3: plots the probability of PCs and monsters winning for a range of PC CV values
diff = 0.7 # deadly

party_hp = 100
pc_r_win = 3
pc_dpr_cvs = np.linspace(0.2, 0.8, 7)


mon_hp = 100
mon_r_win = pc_r_win/diff
mon_dpr_cv = 0.5


party_dpr = mon_hp/pc_r_win
party_size = 4
pc_hp = party_hp/party_size
pc_dpr = party_dpr/party_size

mon_dpr = party_hp/mon_r_win

# original
pc_win_pct = []
mon_win_pct = []

for pc_dpr_cv in pc_dpr_cvs:
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]
    #print(e.encounter_length(units='rounds', method='approx'))


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='PC damage sigma / mean',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='win probability',
            range=[0,1],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=pc_win_pct,
    name=f'PCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'PC CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=mon_win_pct,
    name=f'NPCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'PC CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-pc-cv-deadly-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-pc-cv-deadly-small')

In [55]:
# plots the average damage taken relative to max HP for a range of monster CoV values

diff = 0.7 # deadly

party_hp = 100
party_dpr = 40
party_size = 4
pc_hp = party_hp/4
pc_dpr = party_dpr/4
pc_dpr_cv = 0.8

mon_hp = 100
mon_dpr = (party_hp*diff)/(mon_hp/party_dpr)

# original
pc_win_pct = []
mon_win_pct = []
pc_hp_loss = []
pc_hp_loss_sigma = []
mon_hp_loss = []
mon_hp_loss_sigma = []
mon_dpr_cvs = np.linspace(0.2, 0.8, 7)
for mon_dpr_cv in mon_dpr_cvs:
    #print(f'{mon_dpr_cv:.1f}')
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

    pc_hp_loss += [e.group_damage_mean('NPCs', method='approx', clip=True)/(party_size*pc_hp)]
    pc_hp_loss_sigma += [e.group_damage_sigma('NPCs', method='approx', clip=True)/(party_size*pc_hp)]

    mon_hp_loss += [e.group_damage_mean('PCs', method='approx', clip=True)/(mon_hp)]
    mon_hp_loss_sigma += [e.group_damage_sigma('PCs', method='approx', clip=True)/(mon_hp)]



# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='DPR CV',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='damage / hit points',
            range=[0,1.02],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=pc_hp_loss,
    name=f'PCs',
    #line_color=COLOR_LIST[i],
    error_y=dict(
        type='data',
        symmetric=False,
        array=pc_hp_loss_sigma,
        arrayminus=pc_hp_loss_sigma,
    ),
    hovertemplate=
        f'<b>PCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=mon_hp_loss,
    name=f'NPCs',
    #line_color=COLOR_LIST[i],
    #error_y=dict(
    #    type='data',
    #    symmetric=True,
    #    array=mon_hp_loss_sigma,
    #),
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-example-dt-distribution-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-example-dt-distribution-small')

In [101]:
# Fig. 4: plot DPR CV for published monsters
import numpy as np
import pandas as pd

def baseline_monster(cr):
    return {
        'CR': cr,
        'hit_points': np.round(16*(cr + 1) if cr < 20 else 368 + 48*(cr - 20), 0),
        'damage_per_round': np.round(6*(cr + 1) if cr < 20 else 132 + 12*(cr - 20), 0),
        'attack_bonus': np.round(3.5 + cr/2, 0),
        'save_dc': np.round(11.5 + cr/2, 0),
        'armor_class': np.round(13.0 + cr/3, 0),
        'save_bonus': np.round(0.0 + cr/2, 0),
    }

def monster_dpr_is_valid(monster):
    dpr = 0.65*baseline_monster(monster['CR'])['damage_per_round']
    return monster['dpr.mean']/dpr > 0.5

dM = pd.read_csv('./monster-variability.csv')
dM['dpr.mean'] = dM['damage.mean']/3
dM['dpr.sigma'] = dM['damage.sigma']/np.sqrt(3)
dM['dpr.cv'] = dM['dpr.sigma']/dM['dpr.mean']
dM['valid'] = dM.apply(lambda r: monster_dpr_is_valid(r), axis=1)
dM = dM[dM['valid'] & dM['CR'].between(0.01,30)]

dM = dM[['CR','dpr.cv']].groupby('CR').mean().reset_index()
dM.sort_values(by='CR', inplace=True)

fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='challenge rating',
            range=[0, 27],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='NPC damage sigma / mean',
            range=[-0.01, 1.01],
            tickformat='.1f',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.05),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=0.99, y=0.99,
            orientation='v',
            tracegroupgap=0,
        ),
    )
)

fig.add_trace(go.Scatter(
    x=dM['CR'], 
    y=dM['dpr.cv'],
    name='published monsters',
    mode='markers',
    hovertemplate=
            'Challenge Rating %{x:.0f}<br>'+
            'DPR CV %{y:.2f}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-monster-dpr-cv-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-monster-dpr-cv-small')

In [99]:
# Fig. 5: plots the probability of PCs losing over a range of difficulties for each tier of play

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='probability of PCs losing',
            #range=[-0.01,1.01],
            range=[-0.01,0.51],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.01, y=0.03,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        #line_color=color,
        #marker_color=color,
        fillcolor=color,
        #hoveron = 'points+fills', # select where hover is active
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.47,
        text=name,
        showarrow=False,
    )

tiers = [
    {
        'name': 'tier 1',
        'cv': 0.8,
        'dash': 'solid',
    },
    {
        'name': 'tier 2',
        'cv': 0.6,
        'dash': 'dash',
    },
    {
        'name': 'tier 3',
        'cv': 0.4,
        'dash': 'dashdot',
    },
    {
        'name': 'tier 4',
        'cv': 0.2,
        'dash': 'dashdot',
    },
]

colors = iter(COLOR_LIST)
difficulties = list(np.linspace(0.05, 1.0, 40))
for tier in tiers:
    pc_win_pct = []
    mon_win_pct = []
    for diff in difficulties:
        party_hp = 100
        pc_r_win = 3
        pc_dpr_cv = tier['cv']

        mon_hp = 100
        mon_r_win = pc_r_win/diff
        mon_dpr_cv = tier['cv']


        party_dpr = mon_hp/pc_r_win
        party_size = 4
        pc_hp = party_hp/party_size
        pc_dpr = party_dpr/party_size

        mon_dpr = party_hp/mon_r_win

        # construct combatants
        combatants = [
            cp.Combatant(
                group='NPCs',
                name='monster',
                opponent='PCs',
                level=5,
                hit_points = mon_hp,
                damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 1',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 2',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 3',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 4',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
        ]
        e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
        pc_win_pct += [e.group_win_pct('PCs', method='approx')]
        mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

    # plot data
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=mon_win_pct,
        name=tier['name'],
        #name=f'CV = {mon_dpr_cv:.2f} - NPCs',
        mode='lines',
        line_color=next(colors),
        #line_color=COLOR_LIST[1],
        #line_dash=tier['dash'],
        hovertemplate=
            #f'<b>CV = {mon_dpr_cv:.2f} - NPCs</b><br>'+
            f'<b>' + tier['name'] + '</b><br>'+
            'difficulty %{x:.2f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-diff-tiers-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-diff-tiers-small')

In [98]:
# plots the average damage taken by the PCs and monsters for a range of difficulties

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='damage taken / hit points',
            range=[-0.01,1.01],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.01, y=0.99,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        #line_color=color,
        #marker_color=color,
        fillcolor=color,
        #hoveron = 'points+fills', # select where hover is active
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.5,
        text=name,
        showarrow=False,
    )

tiers = [
    {
        'name': 'tier 1',
        'cv': 0.8,
        'dash': 'solid',
    },
    {
        'name': 'tier 2',
        'cv': 0.6,
        'dash': 'dash',
    },
    {
        'name': 'tier 3',
        'cv': 0.4,
        'dash': 'dashdot',
    },
    {
        'name': 'tier 4',
        'cv': 0.2,
        'dash': 'dashdot',
    },
]

colors = iter(COLOR_LIST)
difficulties = list(np.linspace(0.05, 1.0, 40))
for tier in tiers:
    pc_win_pct = []
    mon_win_pct = []
    pc_hp_loss = []
    pc_hp_loss_sigma = []
    mon_hp_loss = []
    mon_hp_loss_sigma = []
    for diff in difficulties:
        party_hp = 100
        pc_r_win = 3
        pc_dpr_cv = tier['cv']

        mon_hp = 100
        mon_r_win = pc_r_win/diff
        mon_dpr_cv = tier['cv']


        party_dpr = mon_hp/pc_r_win
        party_size = 4
        pc_hp = party_hp/party_size
        pc_dpr = party_dpr/party_size

        mon_dpr = party_hp/mon_r_win

        
        combatants = [
            cp.Combatant(
                group='NPCs',
                name='monster',
                opponent='PCs',
                level=5,
                hit_points = mon_hp,
                damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 1',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 2',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 3',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 4',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
        ]
        e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
        pc_win_pct += [e.group_win_pct('PCs', method='approx')]
        mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

        pc_hp_loss += [e.group_damage_mean('NPCs', method='approx', clip=False)/(party_size*pc_hp)]
        #pc_hp_loss_sigma += [e.group_damage_sigma('NPCs', method='approx', clip=True)/(party_size*pc_hp)]

        #mon_hp_loss += [e.group_damage_mean('PCs', method='approx', clip=True)/(mon_hp)]
        #mon_hp_loss_sigma += [e.group_damage_sigma('PCs', method='approx', clip=True)/(mon_hp)]


    # plot data
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=pc_hp_loss,
        name=tier['name'],
        mode='lines',
        line_color=next(colors),
        hovertemplate=
            f'<b>' + tier['name'] + '</b><br>'+
            'difficulty %{x:.2f}<br>'+
            'HP lost %{y:.1%}<extra></extra>',
    ))

    """fig.add_trace(go.Scatter(
        x=difficulties,
        y=mon_hp_loss,
        name=f'NPCs',
        mode='lines',
        line_color=COLOR_LIST[1],
        hovertemplate=
            f'<b>NPCs</b><br>'+
            'difficulty %{x:.2f}<br>'+
            'HP lost %{y:.1%}<extra></extra>',
    ))"""

fig.add_trace(go.Scatter(
    x=[0,1],
    y=[0,1],
    name=f'1-to-1',
    showlegend=False,
    mode='lines',
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-diff-deadly-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-diff-deadly-small')